# HW2 Documentation

In [1]:
from default import *
import os

## Run the default solution on dev

In [2]:
lexsub = LexSub(os.path.join('data','glove.6B.100d.magnitude'))
output = []
with open(os.path.join('data','input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexsub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))

sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner


## Evaluate the default output

In [3]:
from lexsub_check import precision
with open(os.path.join('data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=27.89


## Functions for retrofit word vector

In [4]:
def norm_word(word):
  if word.lower().isdigit():
    return '-number-'
  elif word.lower() == '':
    return '-punctuation-'
  else:
    return word.lower()

The norm_word function simply normalize a word in the lexicon.

In [5]:
def read_lex(lex_file):
    lexicon = {}
    for line in open(lex_file, 'r'):
        words = line.lower().strip().split()
        lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
    return lexicon

The read_lex function read a lexicon txt file and write it as dictionary for latter use.

In [6]:
def write_wvtxt(wordVec, outFileName):
    sys.stderr.write('\nWriting retrofitted word vectors in '+outFileName+'\n')
    output = open(outFileName, 'w')  
    for word, values in wordVec.items():
        output.write(word+' ')
        for element in wordVec[word]:
            output.write('%.5f' %(element)+' ')
        output.write('\n')      
    output.close()

The write_wvtxt function write the retrofitted new word vector into a new txt file.

In [7]:

def retrofit(self, lexicon, T = 10):
        # initialize retrofit word vector equal to origin word vector
        newwv = {}
        for key, vector in self.wvecs:
            newwv[key] = vector

        loopVocab = []
        for key in lexicon:
            if key in self.wvecs:
                loopVocab.append(key)

        for i in range(T):
            for word in loopVocab:

                wordneighbours = []
                for neighbours in lexicon[word]:
                    if neighbours in self.wvecs:
                        wordneighbours.append(neighbours)
                numneighbours = len(wordneighbours)
                # pass if no neighbours (use original dataset)
                if numneighbours == 0:
                    continue
                # the weight(sum of bij and aij) of the data estimate is the number of neighbours
                newVec= numneighbours * self.wvecs.query(word)

                for qjword in wordneighbours:
                    newVec += newwv[qjword]

                newwv[word] = newVec / (2 * numneighbours)

        return newwv

The retrofit function will take a lexicon dictionary and T = 10(changes in Euclidean distance of adjacent vertices of roughly 10−2) as the parameter. It will first copy the original word vector and generate a retrofitted word vector. The weight of the data estimate(bij and aij in the assignment instructor)is the number of neighbors correspond to a specific word. For the words which do not have any neighbors (no edge connect it with other words), we use the original dataset without retrofit it's word vector.

## Run the retrofitted solution on dev

In [8]:
lexsub = LexSub(os.path.join('data','glove.6B.100d.retrofit.magnitude'))
output = []
with open(os.path.join('data','input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexsub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))

edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along
edge front bottom line corner back left then way along


## Evaluate the retrofitted output

In [9]:
from lexsub_check import precision
with open(os.path.join('data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=40.87


## Substitutes2 function (Incorporating Context Words)

In [10]:
def substitutes2(self, index, sentence):
        substitutability = {}
        target = sentence[index]
        sentential_context = sentence
        guesses = []
        neighbours = list(map(lambda k: k[0], self.wvecs.most_similar(target, topn = 50)))

        # s : lexical substitude (target word's neighbours)
        # t : target substitude word
        # c : words in sentential context 

        for s in neighbours:
            cos_s_t = self.wvecs.similarity(s, target)
            cos_s_c = 0
            for c in sentential_context:
                if c in self.wvecs:
                    cos_s_c += self.wvecs.similarity(s, c)
            substitutability[s] = ( cos_s_t + cos_s_c ) / (1 + len(sentential_context))


        for i in range(10):
            guesses.append(min(substitutability, key=substitutability.get))
            del substitutability[min(substitutability, key=substitutability.get)]


        return guesses 

## Analysis
According to the baseline, we implemented retrofitting by the formulas which we have in the Retrofitting Word Vectors with Semantic Lexicons section.The norm_word function simply normalize a word in the lexicon. The read_lex function read a lexicon txt file and write it as dictionary for latter use. The write_wvtxt function write the retrofitted new word vector into a new txt file. We tried different α and β values. We set α and β to be 1 at first. We got Score=39. Finally, we let α and β equal to the number of neighbors correspond to a specific word. With the functions for retrofit word vector, we improved the accuracy. We got Score=40.87. Then, we tried to incorporate context words. We attempted to use several formula described in the reference material but it did not work well(the above formula uses Add formula to calculate substitutability). No matter which formula we choose, how many candidates are choosen initial, We got a lower accuracy(lower than 40). From the results wo have got, using retrofit function and default substitutes function gives us a better solution.